# 3. Latihan

Gunakan data titanic dari seaborn.  
* target=survived  
* fitur= 'pclass','sex','age','fare'  
'age' tidak perlu pakai dummy variabel, pakai np.where saja



* isi missing value
* split data menjadi train dan test (80:20)
* bandingkan performa model berikut:
    * logistic regresi tanpa scaling
    * Logistic regresi dengan scaling
    * KNN tanpa scaling
    * KNN dengan scaling
    * Decision Tree


In [8]:
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
titanic=sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df=titanic[['survived','pclass','sex','age','fare']]
df

,survived,pclass,sex,age,fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500
...,...,...,...,...,...
886,0,2,male,27.0,13.0000
887,1,1,female,19.0,30.0000
888,0,3,female,NaN,23.4500
889,1,1,male,26.0,30.0000


In [4]:
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
fare          0
dtype: int64

In [5]:
df['age'].fillna(df['age'].median(), inplace=True)

C:\Users\asus\anaconda3\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [6]:
df.isna().sum()

survived    0
pclass      0
sex         0
age         0
fare        0
dtype: int64

In [9]:
df['sex_male'] = np.where(df['sex']=='male',1,0)

<ipython-input-9-7a07fa0b1e03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex_male'] = np.where(df['sex']=='male',1,0)


In [10]:
df.head()

,survived,pclass,sex,age,fare,sex_male
0,0,3,male,22.0,7.2500,1
1,1,1,female,38.0,71.2833,0
2,1,3,female,26.0,7.9250,0
3,1,1,female,35.0,53.1000,0
4,0,3,male,35.0,8.0500,1


In [11]:
target='survived'
fitur=['pclass','age','fare','sex_male']

In [12]:
x=df[fitur]
y=df[target]

In [13]:
x.describe()

,pclass,age,fare,sex_male
count,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.361582,32.204208,0.647587
std,0.836071,13.019697,49.693429,0.477990
min,1.000000,0.420000,0.000000,0.000000
25%,2.000000,22.000000,7.910400,0.000000
50%,3.000000,28.000000,14.454200,1.000000
75%,3.000000,35.000000,31.000000,1.000000
max,3.000000,80.000000,512.329200,1.000000


## 3.1. Logistic Regression
**A. Modelling**

In [14]:
import statsmodels.api as sm

In [15]:
sm_logit= sm.Logit(y,sm.add_constant(x))
result=sm_logit.fit()

Optimization terminated successfully.
         Current function value: 0.452022
         Iterations 6


In [16]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:               survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      886
Method:                           MLE   Df Model:                            4
Date:                Sun, 04 Jul 2021   Pseudo R-squ.:                  0.3212
Time:                        22:47:09   Log-Likelihood:                -402.75
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 3.282e-81
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.6553      0.509      9.153      0.000       3.659       5.652
pclass        -1.1529      0.136     -8.505      0.000      -1.419      -0.887
age           -0.0331      0.007     -4.477      0.0

In [17]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [22]:
def calc_vif(x):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = x.columns
    vif["VIF"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]

    return(vif)

In [23]:
calc_vif(x)

,variables,VIF
0,pclass,3.728285
1,age,4.057797
2,fare,1.426615
3,sex_male,2.901604


**B. Model Interpretation**

In [24]:
#B1 (pclass)
np.exp(-1.1529*1)

0.3157198529193261

In [25]:
#B2 (age)
np.exp(-0.0331*1)

0.9674418105706046

In [26]:
#B3 (fare)
np.exp(0.0006*1)

1.0006001800360054

In [27]:
#B4
np.exp(-2.6073*1)

0.07373335530602235

**C. Validation**
* Train data = 80%
* Test data = 20%

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [29]:
x_train, x_test, y_train, y_test = train_test_split(
    x, 
    y, 
    stratify = y,
    test_size = 0.2, 
    random_state = 2020)

In [30]:
sm_logit_train = sm.Logit(y_train,sm.add_constant(x_train))
result_train = sm_logit_train.fit()

Optimization terminated successfully.
         Current function value: 0.448448
         Iterations 6


In [31]:
 y_predict_proba = result_train.predict(sm.add_constant(x_test))
 y_predict_class = np.where(y_predict_proba>0.5,1,0)

In [32]:
print('model accuracy in test dataset:',accuracy_score(y_test,y_predict_class))
print('Akurasi model 79,3%')

model accuracy in test dataset: 0.7932960893854749
Akurasi model 79,3%


## 3.2. KNN Tanpa Scalling

In [33]:
from sklearn.neighbors import KNeighborsClassifier

In [34]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
y_predict_class = knn.predict(x_test)

In [35]:
print('model accuracy in test dataset:',accuracy_score(y_test,y_predict_class))
print('Akurasi model 72.06%')

model accuracy in test dataset: 0.7206703910614525
Akurasi model 72.06%


## 3.3. KNN Dengan Scalling

In [36]:
from sklearn.preprocessing import MinMaxScaler

In [37]:
scaler = MinMaxScaler()
scaler.fit(x_train) # preprocess fit
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [38]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train_scaled,y_train) # model fit
y_predict_class = knn.predict(x_test_scaled) # model predict

In [39]:
print('model accuracy in test dataset:',accuracy_score(y_test,y_predict_class))
print('Akurasi model 78.21%')

model accuracy in test dataset: 0.7821229050279329
Akurasi model 78.21%


## 3.4. Decision Tree

* Scalling

In [40]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
tree= DecisionTreeClassifier(max_depth=2)
tree.fit(x_train,y_train)
y_predict= tree.predict(x_test)

In [42]:
print('Nilai akurasi model ini:', accuracy_score(y_test,y_predict))

Nilai akurasi model ini: 0.7430167597765364


* Tanpa Scalling

In [43]:
tree=DecisionTreeClassifier(max_depth=2)
tree.fit(x_train_scaled,y_train)
y_predict=tree.predict(x_test_scaled)

In [44]:
print('akurasi setelah scalling=', accuracy_score(y_test,y_predict))

akurasi setelah scalling= 0.7430167597765364
